In [1]:
import functools
from cirkit.symbolic.circuit import Circuit
from cirkit.templates.region_graph import RandomBinaryTree, RegionGraph
from cirkit.symbolic.layers import CategoricalLayer, GaussianLayer
from cirkit.symbolic.parameters import mixing_weight_factory
from cirkit.templates.utils import (
    Parameterization,
    parameterization_to_factory
)

NUM_INPUT_UNITS = 5
NUM_SUM_UNITS = 5

def define_circuit_from_rg(rg: RegionGraph, sum_prod_layer: str = 'cp') -> Circuit:
    # Here is where Overparameterization comes in
    input_factory = lambda scope, num_units: GaussianLayer(
        scope=scope,
        num_output_units=num_units # Overparameterization
    )

    # We need to specify how to parameterize the sum layers
    # Here we choose to initialize them by sampling from a normal distribution and use softmax as activation function.
    sum_weight_param = Parameterization(activation='softmax', initialization='normal')
    sum_weight_factory = parameterization_to_factory(sum_weight_param)

    # Optionally, we can parameterize sum layers receiving input from more than one other layer differently
    # If the below is not specified, then sum_weight_factory will be used.
    # Instead, we choose to parameterize them such that they compute a weighted combinations of the input vectors.
    # Sum layers of this kind are also referred to as mixing layers. In this particular case, sum layers receiving
    # input from more than one layer are parameterized such that they compute a convex combination of the input vectors.
    nary_sum_weight_factory = functools.partial(mixing_weight_factory, param_factory=sum_weight_factory)

    circuit = rg.build_circuit(
        input_factory=input_factory,
        sum_weight_factory=sum_weight_factory,
        nary_sum_weight_factory=nary_sum_weight_factory,
        num_input_units=NUM_INPUT_UNITS,
        num_sum_units=NUM_SUM_UNITS,
        sum_product=sum_prod_layer
    )
    return circuit

In [9]:
from cirkit.templates.region_graph import RandomBinaryTree

from cirkit.pipeline import compile
from cirkit.pipeline import PipelineContext


ctx = PipelineContext(
    backend='torch',      # Use the PyTorch backend
    # Specify the backend compilation flags next
    semiring='lse-sum',   # Use the 'lse-sum' semiring
    fold=True,            # Enable circuit folding
    # -------- Enable layer optimizations -------- #
    optimize=False,
    # -------------------------------------------- #
)

with ctx:
    rnd = RandomBinaryTree(2, depth=None, num_repetitions=1)
    circuit = define_circuit_from_rg(rnd, 'tucker')
    cc = ctx.compile(circuit)
    print(cc)

TorchCircuit(
  (0): TorchGaussianLayer(
    folds: 2  variables: 1  output-units: 5
    input-shape: (2, 1, -1, 1)
    output-shape: (2, -1, 5)
    (mean): TorchParameter(
      shape: (2, 5)
      (0): TorchTensorParameter(output-shape: (2, 5))
    )
    (stddev): TorchParameter(
      shape: (2, 5)
      (0): TorchTensorParameter(output-shape: (2, 5))
      (1): TorchScaledSigmoidParameter(
        input-shapes: [(2, 5)]
        output-shape: (2, 5)
      )
    )
  )
  (1): TorchKroneckerLayer(
    folds: 1  arity: 2  input-units: 5  output-units: 25
    input-shape: (1, 2, -1, 5)
    output-shape: (1, -1, 25)
  )
  (2): TorchSumLayer(
    folds: 1  arity: 1  input-units: 25  output-units: 1
    input-shape: (1, 1, -1, 25)
    output-shape: (1, -1, 1)
    (weight): TorchParameter(
      shape: (1, 1, 25)
      (0): TorchTensorParameter(output-shape: (1, 1, 25))
      (1): TorchSoftmaxParameter(
        input-shapes: [(1, 1, 25)]
        output-shape: (1, 1, 25)
      )
    )
  )
)


In [7]:
from collections import Counter
print(Counter(type(m).__name__ for m in cc.modules()))

Counter({'ModuleList': 4, 'TorchParameter': 3, 'TorchTensorParameter': 3, 'ParameterAddressBook': 3, 'TorchCircuit': 1, 'TorchGaussianLayer': 1, 'TorchScaledSigmoidParameter': 1, 'TorchTuckerLayer': 1, 'TorchSoftmaxParameter': 1, 'LayerAddressBook': 1})
